In [1]:
import numpy as np
import math
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,MinMaxScaler,OrdinalEncoder
from sklearn.feature_extraction import FeatureHasher
import xgboost as xgb
import lightgbm as lgb
import category_encoders as ce
import random
from sklearn.cluster import DBSCAN

In [28]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
y = df_train['Fault']
df_train = df_train.drop('Fault',axis=1)
id = df_test['Id']
df_test = df_test.drop('Id',axis=1)
df_c= pd.concat((df_train,df_test))

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [29]:
def time_features(df,time_col):
    df[time_col] = pd.to_datetime(df[time_col])
    df['dayofweek'] = pd.to_datetime(df[time_col]).dt.dayofweek
    df['hour'] = pd.to_datetime(df[time_col]).dt.hour
    df['month'] = pd.to_datetime(df[time_col]).dt.month
    df = df.drop(time_col,axis=1)
    return df

In [30]:
df_c = df_c.drop('Report Number',axis=1)
df_c = df_c.drop('Local Case Number',axis=1)
df_c['Location'] = df_c['Location'].str.replace(',',' ')
df_c[['l1', 'l2']] = df_c.Location.str.split(' ', expand = True ).astype(float)
df_c.drop(columns = ['Location'], inplace = True)
df_c['l2'] = MinMaxScaler().fit_transform(np.array(df_c['l2']).reshape(-1,1))
df_c = df_c.drop('Municipality',axis=1)
df_c = df_c.drop('Related Non-Motorist',axis=1)
df_c = df_c.drop('Cross-Street Name',axis=1)
df_c = df_c.drop('Cross-Street Type',axis=1)
# df_c['Cross-Street Name'] = df_c['Cross-Street Name'].astype("category").cat.codes
# df_c = df_c.drop('Speed Limit',axis=1)
# df_c = df_c.drop('Vehicle Movement',axis=1)
# df_c = df_c.drop('Vehicle Body Type',axis=1)
df_c = df_c.drop('Parked Vehicle',axis=1)
# df_c = df_c.drop('Vehicle Damage Extent',axis=1)
df_c = df_c.drop('Driverless Vehicle',axis=1)
df_c = df_c.drop('Circumstance',axis=1)
df_c = df_c.drop('Vehicle Make',axis=1)
df_c = df_c.drop('Drivers License State',axis=1)
df_c = df_c.drop('Vehicle Model',axis=1)
df_c = df_c.drop('Person ID',axis=1)
df_c = df_c.drop('Vehicle ID',axis=1)
df_c = df_c.drop('Vehicle Year',axis=1)
df_c = time_features(df_c,'Crash Date/Time')
df_c["dir_change"] = (df_c['Vehicle Going Dir']==df_c['Vehicle Continuing Dir']).astype("int8")
df_c = df_c.drop('Vehicle Going Dir',axis=1)
df_c = df_c.drop('Vehicle Continuing Dir',axis=1)
df_c = df_c.drop('Road Name',axis=1)
df_c = df_c.drop('Off-Road Description',axis=1)
df_c

,Agency Name,ACRS Report Type,Route Type,Collision Type,Weather,Surface Condition,Light,Traffic Control,Driver Substance Abuse,Non-Motorist Substance Abuse,...,Speed Limit,Equipment Problems,Latitude,Longitude,l1,l2,dayofweek,hour,month,dir_change
0,Montgomery County Police,Injury Crash,Maryland (State),SINGLE VEHICLE,CLEAR,WET,DAYLIGHT,NO CONTROLS,NONE DETECTED,NaN,...,40,NO MISUSE,39.263378,-77.344203,3.6383,0.999992,0,16,6,0
1,Montgomery County Police,Injury Crash,County,STRAIGHT MOVEMENT ANGLE,CLEAR,DRY,DAYLIGHT,STOP SIGN,NONE DETECTED,NaN,...,25,NO MISUSE,39.068913,-77.063227,3.0862,0.999991,5,10,2,1
2,Montgomery County Police,Property Damage Crash,Maryland (State),SAME DIRECTION RIGHT TURN,CLEAR,DRY,UNKNOWN,OTHER,NaN,NaN,...,35,NO MISUSE,38.993578,-77.032718,3.9583,0.999992,6,8,6,1
3,Montgomery County Police,Injury Crash,County,OTHER,CLOUDY,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,NaN,...,25,OTHER,39.083480,-77.086933,3.8400,0.066386,4,7,5,1
4,Montgomery County Police,Injury Crash,Maryland (State),SAME DIR REAR END,CLEAR,DRY,DAYLIGHT,TRAFFIC SIGNAL,NONE DETECTED,MEDICATION PRESENT,...,40,NO MISUSE,38.996942,-77.109168,3.9917,0.999987,1,17,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77230,Montgomery County Police,Injury Crash,Maryland (State),HEAD ON,CLEAR,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,NaN,...,45,NO MISUSE,39.116285,-76.963613,3.1250,0.999996,2,13,2,1
77231,Montgomery County Police,Property Damage Crash,NaN,STRAIGHT MOVEMENT ANGLE,CLEAR,NaN,DAYLIGHT,NaN,NONE DETECTED,NaN,...,0,NO MISUSE,39.108410,-76.925858,3.0400,0.080659,0,14,12,1
77232,MONTGOMERY,Injury Crash,County,SAME DIR REAR END,CLEAR,DRY,DARK LIGHTS ON,TRAFFIC SIGNAL,NONE DETECTED,NaN,...,40,NaN,39.122352,-77.177598,3.2317,0.999997,6,19,9,0
77233,Montgomery County Police,Property Damage Crash,Maryland (State),SAME DIRECTION LEFT TURN,CLEAR,DRY,DAYLIGHT,NO CONTROLS,NONE DETECTED,NaN,...,25,NO MISUSE,38.983770,-77.093128,3.8708,0.999999,3,11,5,1


In [31]:
# df = pd.get_dummies(df)
obj=[]
for col in df_c:
    if(df_c[col].dtype=='object'):
        obj.append(col)
encoder = ce.polynomial.PolynomialEncoder(cols=obj)
df_train = df_c.iloc[:51490,:]
df_test = df_c.iloc[51490:,:]
df_train = encoder.fit_transform(df_train)
df_test = encoder.transform(df_test)

In [32]:
df_test

,intercept,Agency Name_0,Agency Name_1,Agency Name_2,Agency Name_3,Agency Name_4,Agency Name_5,Agency Name_6,Agency Name_7,Agency Name_8,...,Equipment Problems_6,Equipment Problems_7,Latitude,Longitude,l1,l2,dayofweek,hour,month,dir_change
0,1,-0.275241,-0.087039,0.377854,-0.317882,-0.035806,0.389249,-0.503518,0.373979,-0.163266,...,-0.034139,0.008815,38.988440,-77.127668,3.8400,0.051451,2,17,12,1
1,1,-0.495434,0.522233,-0.453425,0.336581,-0.214834,0.116775,-0.052694,0.018699,-0.004535,...,-0.034139,0.008815,39.094075,-77.205783,3.9050,0.999988,6,19,5,0
2,1,-0.495434,0.522233,-0.453425,0.336581,-0.214834,0.116775,-0.052694,0.018699,-0.004535,...,-0.034139,0.008815,39.047180,-77.052688,3.4100,0.071011,4,14,3,1
3,1,-0.495434,0.522233,-0.453425,0.336581,-0.214834,0.116775,-0.052694,0.018699,-0.004535,...,0.477952,-0.493627,39.178007,-77.238475,3.7067,0.999991,4,17,5,1
4,1,-0.495434,0.522233,-0.453425,0.336581,-0.214834,0.116775,-0.052694,0.018699,-0.004535,...,-0.477952,0.246813,39.056134,-77.154043,3.5143,0.999993,5,17,9,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77230,1,-0.495434,0.522233,-0.453425,0.336581,-0.214834,0.116775,-0.052694,0.018699,-0.004535,...,-0.034139,0.008815,39.116285,-76.963613,3.1250,0.999996,2,13,2,1
77231,1,-0.495434,0.522233,-0.453425,0.336581,-0.214834,0.116775,-0.052694,0.018699,-0.004535,...,-0.034139,0.008815,39.108410,-76.925858,3.0400,0.080659,0,14,12,1
77232,1,-0.275241,-0.087039,0.377854,-0.317882,-0.035806,0.389249,-0.503518,0.373979,-0.163266,...,0.477952,-0.493627,39.122352,-77.177598,3.2317,0.999997,6,19,9,0
77233,1,-0.495434,0.522233,-0.453425,0.336581,-0.214834,0.116775,-0.052694,0.018699,-0.004535,...,-0.034139,0.008815,38.983770,-77.093128,3.8708,0.999999,3,11,5,1


In [33]:
X = np.array(df_train,dtype=float)
y = np.array(y,dtype=int)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_test = np.array(df_test,dtype=float)

In [57]:
from sklearn.ensemble import StackingClassifier
from catboost import CatBoostClassifier
estimators = [
    ("rf", RandomForestClassifier(n_estimators=1000)),
    ("lgbm", lgb.LGBMClassifier(boosting='dart',n_estimators=1700,drop_rate=0.37,max_drop=65,learning_rate=0.05,reg_alpha=0.1)),
    ("cat", CatBoostClassifier(iterations=1000))
]
clf5 = StackingClassifier(estimators=estimators)
clf5.fit(X,y)
print(clf5.score(X,y))
print(clf5.score(X_val,y_val))

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
Learning rate set to 0.068031
0:	learn: 0.6467932	total: 18.3ms	remaining: 14.6s
1:	learn: 0.6084250	total: 32.2ms	remaining: 12.8s
2:	learn: 0.5776165	total: 44ms	remaining: 11.7s
3:	learn: 0.5519906	total: 57.9ms	remaining: 11.5s
4:	learn: 0.5282411	total: 69.6ms	remaining: 11.1s
5:	learn: 0.5097558	total: 83.6ms	remaining: 11.1s
6:	learn: 0.4936135	total: 97ms	remaining: 11s
7:	learn: 0.4780682	total: 111ms	remaining: 11s
8:	learn: 0.4659406	total: 123ms	remaining: 10.8s
9:	learn: 0.4539856	total: 135ms	remaining: 10.7s
10:	learn: 0.4446436	total: 149ms	remaining: 10.7s
11:	learn: 0.4361079	total: 165ms	remaining: 10.8s
12:	learn: 0.4279646	total: 178ms	remaining: 10.7s
13:	learn: 0.4210876	total: 189ms	remaining: 10.6s
14:	learn: 0.4150490	total: 203ms	remaining: 10.6s
15:	learn: 0.4099320	total: 216ms	remaining: 10.6s
16:	learn: 0.4057062	total: 230ms	remaining: 10.6s
17:	le

In [58]:
X_test = np.array(df_test)
y_pred5=clf5.predict(X_test)
y_pred5=y_pred5.astype(int)
print(np.sum(y_pred5))
data = {'Id':id,'Fault':y_pred5}
ans = pd.DataFrame(data)
ans.to_csv('subn5.csv',index=False)
ans

41709


,Id,Fault
0,0,1
1,1,0
2,2,1
3,3,1
4,4,1
...,...,...
77230,77230,0
77231,77231,1
77232,77232,0
77233,77233,1
